# IMPORT LIBRARIES

In [1]:
import pandas as pd
import random
from pyomo.environ import *
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

In [ ]:
# 1. Build the binary order–SKU matrix R
def build_R(df, order_col='Num. Ordine', sku_col='Articolo'):
    df_unique = df[[order_col, sku_col]].drop_duplicates()
    orders = df_unique[order_col].unique()
    skus = df_unique[sku_col].unique()
    order_idx = pd.Series(np.arange(len(orders)), index=orders)
    sku_idx = pd.Series(np.arange(len(skus)), index=skus)
    rows = df_unique[order_col].map(order_idx).values
    cols = df_unique[sku_col].map(sku_idx).values
    data = np.ones(len(df_unique), dtype=int)
    R = coo_matrix((data, (rows, cols)), shape=(len(orders), len(skus)))
    return R, orders, skus

In [ ]:
# 2. Define the greedy heuristic with debug option
def greedy_orders_volume(R, k_volume, sku_volumes, debug=False):
    R_csr = R.tocsr()
    n_orders, n_skus = R_csr.shape
    n_dcs = len(k_volume)
    skus_per_order = np.diff(R_csr.indptr)
    sorted_orders = np.argsort(-skus_per_order)
    sorted_dcs = np.argsort(-k_volume)
    if debug:
        print("Sorted DCs by capacity:", sorted_dcs)
    allocation = [set() for _ in range(n_dcs)]
    remaining = k_volume.copy()
    # Phase 1
    for o in sorted_orders:
        skus_in_order = R_csr[o].indices
        for s in skus_in_order:
            for d in sorted_dcs:
                if remaining[d] >= sku_volumes[s] and s not in allocation[d]:
                    allocation[d].add(s)
                    remaining[d] -= sku_volumes[s]
                    if debug:
                        print(f"Order {o}: SKU {sku_ids[s]} -> DC_{d+1} (rem {remaining[d]:.2f} m3)")
                    break
    # Phase 2
    C = (R_csr.T).dot(R_csr).toarray()
    # Phase 3
    if debug:
        print("\nFilling remaining capacity by co-appearance")
    while True:
        allocated = False
        for d in sorted_dcs:
            cap = remaining[d]
            candidates = [s for s in range(n_skus) 
                          if s not in allocation[d] and sku_volumes[s] <= cap]
            if not candidates:
                continue
            scores = (C[list(allocation[d]), :].sum(axis=0) 
                      if allocation[d] else C.sum(axis=0))
            mask = np.full(n_skus, -np.inf)
            mask[candidates] = scores[candidates]
            s_best = int(np.argmax(mask))
            allocation[d].add(s_best)
            remaining[d] -= sku_volumes[s_best]
            allocated = True
            if debug:
                print(f"DC_{d+1}: added SKU {sku_ids[s_best]} (score {scores[s_best]:.0f}), rem {remaining[d]:.2f} m3")
        if not allocated:
            break
    return allocation, remaining

# IMPORT DATA

In [5]:
df = pd.read_csv(r'C:\Users\Matteo.Gabellini\OneDrive - Alma Mater Studiorum Università di Bologna\DOTTORATO\1.RICERCA\0.CONFERENCE PAPER\6.ICIL\1.WAREHOUSE ALLOCATION\0.DATA\DatasetClean.csv')
df['Articolo'] = df['Articolo'].astype(str)
df

,Unnamed: 0,Num. Ordine,Mese-Giorno,Articolo,Descrizione,Pezzi ordinati,Pezzi evasi,Pz x CT,Pz x TH,Volume pezzo,Volume cartone,Ecr1,Ecr2,Ecr3,Ecr4,Canale,Cliente,PV,Percorso
0,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,33.96900,Cura Persona,Igienico Sanitari,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26
1,1,738379,04-18,3456,STUDIO L.5 INVISI FIX GEL FOR.LIQ.150ml,6,6,6,0,0.378000,2.66000,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
2,2,738379,04-18,7199,STUDIO L.8 FIX&FORCE GEL IPERFOR.150 ML.,6,6,6,0,0.303750,2.94400,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
3,3,738379,04-18,43556,STUDIO L.9 INDESTRUC.GEL ESTREMO 150 ML,6,6,6,0,0.720000,2.81600,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
4,4,738379,04-18,50045,STUDIO L.5 INVISI FIX GEL CR.FOR.VAS.150,6,6,6,0,0.405000,2.54375,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447292,2447292,791134,12-29,92720,CHANTECLAIR SGRASS.600 MLx2 PZ.LIMONE,1,1,6,0,3.271125,23.04000,Casa e Bucato,Superfici,Detergenti Superfici,Sgrassatori,B2C,2152842,UD,999
2447293,2447293,791134,12-29,98819,VIM GEL BAGNO 5in1 ANTICALCARE 1000 ML,1,1,12,0,1.559250,25.63600,Casa e Bucato,Bagno,Detergenti Bagno e WC,Detergenti Bagno,B2C,2152842,UD,999
2447294,2447294,791134,12-29,105877,CIF CREMA GREEN ACTIVE 500 ML PINK BLOOM,1,1,16,0,0.728000,17.02400,Casa e Bucato,Superfici,Detergenti Superfici,Detergenti Multiuso,B2C,2152842,UD,999
2447295,2447295,791134,12-29,107171,OMINO B.DET.IDROCAPS SALVAFIBRE 20pz,1,1,8,0,2.025000,20.46000,Casa e Bucato,Bucato,Detersivi Bucato,Detersivi Capsule Lavatrice,B2C,2152842,UD,999


In [6]:
df_vol = df.groupby('Articolo')[['Volume pezzo']].median().reset_index()
df_vol

,Articolo,Volume pezzo
0,100002,0.060000
1,100003,0.060000
2,100004,0.060000
3,100005,0.060000
4,100006,0.060000
...,...,...
18655,99972,0.059375
18656,99973,0.270000
18657,99974,3.312000
18658,99975,0.084000


In [7]:
df_stock = pd.read_excel(r'C:\Users\Matteo.Gabellini\OneDrive - Alma Mater Studiorum Università di Bologna\DOTTORATO\1.RICERCA\0.CONFERENCE PAPER\6.ICIL\1.WAREHOUSE ALLOCATION\0.DATA\Giacenza media articoli 2024.xlsx')
df_stock['ARTICOLO'] = df_stock['ARTICOLO'].astype(str)
df_stock = df_stock.groupby('ARTICOLO')[['GIACENZA MEDIA']].mean().reset_index()
df_stock

,ARTICOLO,GIACENZA MEDIA
0,100002,1057.6
1,100003,735.4
2,100004,1072.4
3,100005,1170.0
4,100006,1232.2
...,...,...
20919,99972,19.2
20920,99973,26.5
20921,99974,13.9
20922,99975,7.0


In [8]:
# Perform the join based on 'Articolo'
df_stock_vol = df_stock.merge(df_vol[['Articolo','Volume pezzo']], how='left', left_on='ARTICOLO', right_on='Articolo')

# Drop duplicate column 'ARTICOLO' after merge
df_stock_vol.drop(columns=['Articolo'], inplace=True)

#Compute stock in volum
df_stock_vol['Giacenza Pezzi Volume [m3]'] = df_stock_vol['Volume pezzo'] / 1000
df_stock_vol['Giacenza Pezzi Volume [m3]'] = df_stock_vol['GIACENZA MEDIA'] * df_stock_vol['Giacenza Pezzi Volume [m3]']

#df_stock_vol = df_stock_vol.drop_duplicates()

df_stock_vol

,ARTICOLO,GIACENZA MEDIA,Volume pezzo,Volume pezzo [m3],Giacenza Pezzi Volume [m3]
0,100002,1057.6,0.060000,0.000060,0.063456
1,100003,735.4,0.060000,0.000060,0.044124
2,100004,1072.4,0.060000,0.000060,0.064344
3,100005,1170.0,0.060000,0.000060,0.070200
4,100006,1232.2,0.060000,0.000060,0.073932
...,...,...,...,...,...
20919,99972,19.2,0.059375,0.000059,0.001140
20920,99973,26.5,0.270000,0.000270,0.007155
20921,99974,13.9,3.312000,0.003312,0.046037
20922,99975,7.0,0.084000,0.000084,0.000588


In [9]:
df_stock_vol['Giacenza Pezzi Volume [m3]'].sum()

9121.832503351417

In [10]:
# Perform a left join to maintain the original number of rows in df
df = df.merge(df_stock_vol[['ARTICOLO', 'Giacenza Pezzi Volume [m3]']], how='left', left_on='Articolo', right_on='ARTICOLO')

df['Volume evaso [m3]'] = df['Pezzi evasi'] * df['Volume pezzo'] / 1000

# Drop the extra 'ARTICOLO' column from df_stock_vol (after the merge)
df.drop(columns=['ARTICOLO'], inplace=True)

# Ensure no additional duplicates were introduced
#df = df.drop_duplicates()

df

,Unnamed: 0,Num. Ordine,Mese-Giorno,Articolo,Descrizione,Pezzi ordinati,Pezzi evasi,Pz x CT,Pz x TH,Volume pezzo,...,Ecr1,Ecr2,Ecr3,Ecr4,Canale,Cliente,PV,Percorso,Giacenza Pezzi Volume [m3],Volume evaso [m3]
0,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,...,Cura Persona,Igienico Sanitari,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26,3.474766,0.007728
1,1,738379,04-18,3456,STUDIO L.5 INVISI FIX GEL FOR.LIQ.150ml,6,6,6,0,0.378000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.206955,0.002268
2,2,738379,04-18,7199,STUDIO L.8 FIX&FORCE GEL IPERFOR.150 ML.,6,6,6,0,0.303750,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.140424,0.001823
3,3,738379,04-18,43556,STUDIO L.9 INDESTRUC.GEL ESTREMO 150 ML,6,6,6,0,0.720000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.419256,0.004320
4,4,738379,04-18,50045,STUDIO L.5 INVISI FIX GEL CR.FOR.VAS.150,6,6,6,0,0.405000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.148149,0.002430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447292,2447292,791134,12-29,92720,CHANTECLAIR SGRASS.600 MLx2 PZ.LIMONE,1,1,6,0,3.271125,...,Casa e Bucato,Superfici,Detergenti Superfici,Sgrassatori,B2C,2152842,UD,999,2.330022,0.003271
2447293,2447293,791134,12-29,98819,VIM GEL BAGNO 5in1 ANTICALCARE 1000 ML,1,1,12,0,1.559250,...,Casa e Bucato,Bagno,Detergenti Bagno e WC,Detergenti Bagno,B2C,2152842,UD,999,0.943970,0.001559
2447294,2447294,791134,12-29,105877,CIF CREMA GREEN ACTIVE 500 ML PINK BLOOM,1,1,16,0,0.728000,...,Casa e Bucato,Superfici,Detergenti Superfici,Detergenti Multiuso,B2C,2152842,UD,999,0.743142,0.000728
2447295,2447295,791134,12-29,107171,OMINO B.DET.IDROCAPS SALVAFIBRE 20pz,1,1,8,0,2.025000,...,Casa e Bucato,Bucato,Detersivi Bucato,Detersivi Capsule Lavatrice,B2C,2152842,UD,999,0.532170,0.002025


In [11]:
df.drop_duplicates(subset = 'Articolo').groupby('Ecr1')['Giacenza Pezzi Volume [m3]'].sum().sum()

8737.318431127718

# GREEDY ORDER

In [32]:
# 3. Execute the full pipeline

# Assume df and df_stock_vol are already loaded in the environment
R, order_ids, sku_ids = build_R(df)

# Compute SKU volumes (m3) from df_stock_vol
vol_series = df_stock_vol.set_index('ARTICOLO')['Giacenza Pezzi Volume [m3]']
median_vol = vol_series.median()
vol_series_filled = vol_series.fillna(median_vol)
sku_volumes = np.array([vol_series_filled.get(sku, median_vol) for sku in sku_ids], dtype=float)

# Define DC capacities (m3)
capacity_A = 25000 * 0.37  # replace with actual if needed
capacity_B =  6800 * 0.37
k_volume = np.array([capacity_A, capacity_B])

# Run heuristic with debug on
allocation, remaining_volume = greedy_orders_volume(R, k_volume, sku_volumes, debug=True)
print("\nFinal remaining volumes (m3):", remaining_volume)

# 4. Map indices back to Articolo codes
alloc_map = []
for d, skus in enumerate(allocation):
    for s in skus:
        alloc_map.append({
            'Warehouse': f'DC_{d+1}', 
            'Articolo': sku_ids[s]
        })
df_allocation = pd.DataFrame(alloc_map)

Sorted DCs by capacity: [0 1]
Order 39429: SKU 74657 -> DC_1 (rem 9249.89 m3)
Order 39429: SKU 59758 -> DC_1 (rem 9249.71 m3)
Order 39429: SKU 104758 -> DC_1 (rem 9249.67 m3)
Order 39429: SKU 80786 -> DC_1 (rem 9249.61 m3)
Order 39429: SKU 61329 -> DC_1 (rem 9249.54 m3)
Order 39429: SKU 15269 -> DC_1 (rem 9249.28 m3)
Order 39429: SKU 100964 -> DC_1 (rem 9249.26 m3)
Order 39429: SKU 98838 -> DC_1 (rem 9249.13 m3)
Order 39429: SKU 42523 -> DC_1 (rem 9248.24 m3)
Order 39429: SKU 94320 -> DC_1 (rem 9237.43 m3)
Order 39429: SKU 100644 -> DC_1 (rem 9237.18 m3)
Order 39429: SKU 94793 -> DC_1 (rem 9236.93 m3)
Order 39429: SKU 91270 -> DC_1 (rem 9236.89 m3)
Order 39429: SKU 99320 -> DC_1 (rem 9236.85 m3)
Order 39429: SKU 96874 -> DC_1 (rem 9232.85 m3)
Order 39429: SKU 101260 -> DC_1 (rem 9181.55 m3)
Order 39429: SKU 9057 -> DC_1 (rem 9177.80 m3)
Order 39429: SKU 92596 -> DC_1 (rem 9177.60 m3)
Order 39429: SKU 92594 -> DC_1 (rem 9177.49 m3)
Order 39429: SKU 100944 -> DC_1 (rem 9172.63 m3)
Order 

In [33]:
df_allocation

,Warehouse,Articolo
0,DC_1,20918
1,DC_1,3456
2,DC_1,7199
3,DC_1,43556
4,DC_1,50045
...,...,...
20296,DC_2,94796
20297,DC_2,21077
20298,DC_2,79635
20299,DC_2,104242


# STATISTICS

In [34]:
df = pd.merge(df, df_allocation, on='Articolo', how='left')
df['Warehouse'] = df['Warehouse'].replace({'DC_1': 'A', 'DC_2': 'B'})
df

,Unnamed: 0,Num. Ordine,Mese-Giorno,Articolo,Descrizione,Pezzi ordinati,Pezzi evasi,Pz x CT,Pz x TH,Volume pezzo,...,Ecr3,Ecr4,Canale,Cliente,PV,Percorso,Giacenza Pezzi Volume [m3],Volume evaso [m3],DC,Warehouse
0,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,...,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26,3.474766,0.007728,DC_1,A
1,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,...,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26,3.474766,0.007728,DC_1,B
2,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,...,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26,3.474766,0.007728,DC_2,A
3,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,...,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26,3.474766,0.007728,DC_2,B
4,1,738379,04-18,3456,STUDIO L.5 INVISI FIX GEL FOR.LIQ.150ml,6,6,6,0,0.378000,...,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.206955,0.002268,DC_1,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5048133,2447292,791134,12-29,92720,CHANTECLAIR SGRASS.600 MLx2 PZ.LIMONE,1,1,6,0,3.271125,...,Detergenti Superfici,Sgrassatori,B2C,2152842,UD,999,2.330022,0.003271,DC_1,A
5048134,2447293,791134,12-29,98819,VIM GEL BAGNO 5in1 ANTICALCARE 1000 ML,1,1,12,0,1.559250,...,Detergenti Bagno e WC,Detergenti Bagno,B2C,2152842,UD,999,0.943970,0.001559,DC_1,A
5048135,2447294,791134,12-29,105877,CIF CREMA GREEN ACTIVE 500 ML PINK BLOOM,1,1,16,0,0.728000,...,Detergenti Superfici,Detergenti Multiuso,B2C,2152842,UD,999,0.743142,0.000728,DC_1,A
5048136,2447295,791134,12-29,107171,OMINO B.DET.IDROCAPS SALVAFIBRE 20pz,1,1,8,0,2.025000,...,Detersivi Bucato,Detersivi Capsule Lavatrice,B2C,2152842,UD,999,0.532170,0.002025,DC_1,A


In [35]:
df_results = pd.DataFrame.from_dict(
    {
        'Magazzino A': [],
        'Magazzino B':[],
        
        'Codici in A':[],
        'Codici in B':[],
        
        'Stock [m3] in A':[],
        'Stock [m3] in B':[],
        
        '% Ordini completati in AB':[],
        '% Ordini completati in A':[], 
        '% Ordini completati in B':[],

        'Vol[m3] Ordini completati in A':[], 
        'Vol[m3] Ordini completati in B':[],
        'Vol[m3] Ordini completati in AB':[],
        'Vol[m3] Ordini completati in AB (A)':[],
        'Vol[m3] Ordini completati in AB (B)':[],
        
        '% Rotte completate in AB' :[],
        '% Rotte completate in A' :[],
        '% Rotte completate in B' :[],

        'Vol[m3] Rotte completati in A' :[],
        'Vol[m3] Rotte completati in B' :[],
        'Vol[m3] Rotte completati in AB' :[],
        'Vol[m3] Rotte completati in AB (A)' :[],
        'Vol[m3] Rotte completati in AB (B)' :[],

    }
)
df_results

,Magazzino A,Magazzino B,Codici in A,Codici in B,Stock [m3] in A,Stock [m3] in B,% Ordini completati in AB,% Ordini completati in A,% Ordini completati in B,Vol[m3] Ordini completati in A,...,Vol[m3] Ordini completati in AB (A),Vol[m3] Ordini completati in AB (B),% Rotte completate in AB,% Rotte completate in A,% Rotte completate in B,Vol[m3] Rotte completati in A,Vol[m3] Rotte completati in B,Vol[m3] Rotte completati in AB,Vol[m3] Rotte completati in AB (A),Vol[m3] Rotte completati in AB (B)


In [36]:
assignment_A = df.groupby('Warehouse')['Articolo'].unique().get('A', 0)
assignment_B = df.groupby('Warehouse')['Articolo'].unique().get('B', 0)

code_A = df.groupby('Warehouse')['Articolo'].nunique().get('A', 0)
code_B = df.groupby('Warehouse')['Articolo'].nunique().get('B', 0)
# print('Article division', article_division)

#Order analysis
order_grouped_df = df.groupby(['Mese-Giorno','Num. Ordine']).agg({
    'Warehouse': lambda x: list(x.unique()),  # Stores unique warehouses as lists
    'Volume evaso [m3]': 'sum'  # Sums up volume
}).reset_index()

order_movment_A = len(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(order_grouped_df) * 100
order_movment_B = len(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]) / len(order_grouped_df) * 100
order_movment_AB = len(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(order_grouped_df) * 100

order_vol_A = order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum() 
order_vol_B = order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]['Volume evaso [m3]'].sum() 
order_vol_AB = order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum()

AB_order_list = list(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Num. Ordine'])
df_AB_order_volume = df.groupby(['Num. Ordine','Warehouse'])[['Volume evaso [m3]']].sum().reset_index()
df_AB_order_volume = df_AB_order_volume[df_AB_order_volume['Num. Ordine'].isin(AB_order_list)].groupby('Warehouse')['Volume evaso [m3]'].sum()

#Route analysis
route_grouped_df = df.groupby(['Mese-Giorno','Percorso']).agg({
    'Warehouse': lambda x: list(x.unique()),  # Stores unique warehouses as lists
    'Volume evaso [m3]': 'sum'  # Sums up volume
}).reset_index()

route_movment_A = len(route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(route_grouped_df) * 100
route_movment_B = len(route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]) / len(route_grouped_df) * 100
route_movment_AB = len(route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(route_grouped_df) * 100

route_vol_A = route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum() 
route_vol_B = route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]['Volume evaso [m3]'].sum() 
route_vol_AB = route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum()

# Step 1: Extract valid routes containing both 'A' and 'B' in 'Warehouse'
AB_order_list = route_grouped_df[
    route_grouped_df['Warehouse'].astype(str).str.contains(r'A') & 
    route_grouped_df['Warehouse'].astype(str).str.contains(r'B')
][['Mese-Giorno', 'Percorso']].apply(tuple, axis=1).tolist()

# Step 2: Compute total volume for each route
df_AB_route_volume = df.groupby(['Mese-Giorno', 'Percorso', 'Warehouse'])[['Volume evaso [m3]']].sum().reset_index()

# Step 3: Filter only the relevant routes and sum by Warehouse
df_AB_route_volume = df_AB_route_volume[
    df_AB_route_volume[['Mese-Giorno', 'Percorso']].apply(tuple, axis=1).isin(AB_order_list)
].groupby('Warehouse')['Volume evaso [m3]'].sum()


weighted_stock = df.groupby(['Articolo','Warehouse',])['Giacenza Pezzi Volume [m3]'].mean() * (df.groupby(['Articolo','Warehouse',])['Pezzi evasi'].sum() / df.groupby(['Articolo'])['Pezzi evasi'].sum())
stock_A = weighted_stock.groupby('Warehouse').sum().get('A', 0)
stock_B = weighted_stock.groupby('Warehouse').sum().get('B', 0)

df_results = pd.DataFrame.from_dict(
    {
        'Magazzino A': [assignment_A],
        'Magazzino B': [assignment_B],
        
        'Codici in A':[code_A],
        'Codici in B':[code_B],
        
        'Stock [m3] in A':[stock_A],
        'Stock [m3] in B':[stock_B],
        
        '% Ordini completati in AB':[order_movment_AB],
        '% Ordini completati in A':[order_movment_A],
        '% Ordini completati in B':[order_movment_B],

        '% Rotte completate in AB':[route_movment_AB],
        '% Rotte completate in A':[route_movment_A],
        '% Rotte completate in B':[route_movment_B],

        'Vol[m3] Ordini completati in A':[order_vol_A],
        'Vol[m3] Ordini completati in B':[order_vol_B],
        'Vol[m3] Ordini completati in AB':[order_vol_AB],
        'Vol[m3] Ordini completati in AB (A)':[df_AB_order_volume.get('A', 0)],
        'Vol[m3] Ordini completati in AB (B)':[df_AB_order_volume.get('B', 0)],


        'Vol[m3] Rotte completati in A':[route_vol_A],
        'Vol[m3] Rotte completati in B':[route_vol_B],
        'Vol[m3] Rotte completati in AB':[route_vol_AB],
        'Vol[m3] Rotte completati in AB (A)' :[df_AB_route_volume.get('A', 0)],
        'Vol[m3] Rotte completati in AB (B)' :[df_AB_route_volume.get('B', 0)],

        
    }
)

In [37]:
df_results

,Magazzino A,Magazzino B,Codici in A,Codici in B,Stock [m3] in A,Stock [m3] in B,% Ordini completati in AB,% Ordini completati in A,% Ordini completati in B,% Rotte completate in AB,...,Vol[m3] Ordini completati in A,Vol[m3] Ordini completati in B,Vol[m3] Ordini completati in AB,Vol[m3] Ordini completati in AB (A),Vol[m3] Ordini completati in AB (B),Vol[m3] Rotte completati in A,Vol[m3] Rotte completati in B,Vol[m3] Rotte completati in AB,Vol[m3] Rotte completati in AB (A),Vol[m3] Rotte completati in AB (B)
0,"[20918, 3456, 7199, 43556, 50045, 74657, 27576...","[20918, 3456, 43556, 50045, 59758, 80786, 9686...",18660,1641,7862.718668,1257.999993,65.829113,34.170887,0.0,88.247117,...,2303.733008,0.0,95970.12808,60557.70962,35412.41846,447.08192,0.0,97826.779167,62414.360707,35412.41846


In [38]:
df_results.to_excel(r'C:\Users\Matteo.Gabellini\OneDrive - Alma Mater Studiorum Università di Bologna\DOTTORATO\1.RICERCA\0.CONFERENCE PAPER\6.ICIL\1.WAREHOUSE ALLOCATION\3.RESULTS\CATELAN_1.xlsx')